In [1]:
input = open("covid19.txt")
lines = [line.rstrip() for line in input]

In [2]:
genome = ''.join(lines[1:])

In [3]:
def pattern_count(text, pattern):
    count = 0
    for i in range(len(text) - len(pattern)):
        if text[i:i+len(pattern)] == pattern:
            count += 1
    return count

In [35]:
def freq_words(text, k):
    result = set()
    cnt = []
    
    for i in range(len(text) - k):
        pattern = text[i:i+k]
        cnt.append(pattern_count(text, pattern))
    
    max_cnt = max(cnt)
    
    for i in range(len(text) - k):
        if cnt[i] == max_cnt:
            result.add(text[i:i+k])
            
    return list(result)

In [37]:
freq_words(genome, 3)

['TTT']

In [38]:
freq_words(genome, 4)

['TGTT']

In [39]:
freq_words(genome, 5)

['TTGTT']

In [40]:
freq_words(genome, 6)

['TTGTTA']

In [41]:
freq_words(genome, 7)

['TTGTTAA']

In [42]:
freq_words(genome, 8)

['ATGGTGTT']

In [43]:
freq_words(genome, 9)

['TAAACGAAC']

In [61]:
def _3_2(text):
    lst = []
    for i in range(len(text) - 7):
        print(text[i:i+3])
        lst.append(text[i:i+3] + "-" + text[i+3+2:i+4+1+3])
    return lst

In [62]:
_3_2("TAATGCCATGGGATGTT")

TAA
AAT
ATG
TGC
GCC
CCA
CAT
ATG
TGG
GGG


['TAA-CCA',
 'AAT-CAT',
 'ATG-ATG',
 'TGC-TGG',
 'GCC-GGG',
 'CCA-GGA',
 'CAT-GAT',
 'ATG-ATG',
 'TGG-TGT',
 'GGG-GTT']